In [26]:
from datasets import load_dataset

In [31]:
ds_gen = load_dataset("brimmann2/squad-v2-sampled2-gen")

In [29]:
from huggingface_hub import notebook_login
notebook_login()

In [32]:
ds = load_dataset("brimmann2/squad-v2-sampled3")

In [37]:
ds

DatasetDict({
    train: Dataset({
        features: ['gem_id', 'id', 'title', 'context', 'question', 'target', 'references', 'answers', 'prompt', 'input_ids', 'attention_mask', 'answers_generated'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['gem_id', 'id', 'title', 'context', 'question', 'target', 'references', 'answers'],
        num_rows: 400
    })
})

In [34]:
ds["train"] = ds_gen["train"]

In [38]:
dval = load_dataset("brimmann2/squad-v2-sampled2-gen2", split="validation")

README.md:   0%|          | 0.00/778 [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/652k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [39]:
dval

Dataset({
    features: ['gem_id', 'id', 'title', 'context', 'question', 'target', 'references', 'answers', 'prompt', 'input_ids', 'attention_mask', 'answers_generated'],
    num_rows: 400
})

In [40]:
ds["validation"] = dval

In [48]:
tf = ds["train"].features

In [47]:
vf = ds["validation"].features

In [50]:
tf

{'gem_id': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None),
 'references': [Value(dtype='string', id=None)],
 'answers': {'text': [Value(dtype='string', id=None)],
  'answer_start': [Value(dtype='int32', id=None)]},
 'prompt': Value(dtype='string', id=None),
 'input_ids': [Value(dtype='int32', id=None)],
 'attention_mask': [Value(dtype='int8', id=None)],
 'answers_generated': Value(dtype='string', id=None)}

In [51]:
vf

{'gem_id': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None),
 'references': [Value(dtype='string', id=None)],
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'prompt': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'answers_generated': Value(dtype='string', id=None)}

In [56]:
from datasets import DatasetDict
new_ds = DatasetDict({
    "train": ds["train"].select_columns(["context", "question", "answers", "answers_generated"]),
    "validation": ds["validation"].select_columns(["context", "question", "answers", "answers_generated"])
})

In [59]:
new_ds

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'answers_generated'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'answers_generated'],
        num_rows: 400
    })
})

In [58]:
new_ds.push_to_hub("brimmann2/squad-v2-sampled4")

ValueError: All datasets in `DatasetDict` should have the same features but features for 'train' and 'validation' don't match: {'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': {'text': [Value(dtype='string', id=None)], 'answer_start': [Value(dtype='int32', id=None)]}, 'answers_generated': Value(dtype='string', id=None)} != {'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None), 'answers_generated': Value(dtype='string', id=None)}

In [60]:
sampled2_ds = load_dataset("brimmann2/squad-v2-sampled3")

In [63]:
sampled2_ds

DatasetDict({
    train: Dataset({
        features: ['gem_id', 'id', 'title', 'context', 'question', 'target', 'references', 'answers'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['gem_id', 'id', 'title', 'context', 'question', 'target', 'references', 'answers'],
        num_rows: 400
    })
})

In [61]:
sampled2_ds["train"].features

{'gem_id': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None),
 'references': [Value(dtype='string', id=None)],
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [62]:
sampled2_ds["validation"].features

{'gem_id': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None),
 'references': [Value(dtype='string', id=None)],
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [69]:
new_new_ds = new_ds["train"].remove_columns("answers")

In [71]:
new_new_new_ds = new_new_ds.add_column("answers", sampled2_ds["train"]["answers"])

In [72]:
new_new_new_ds

Dataset({
    features: ['context', 'question', 'answers_generated', 'answers'],
    num_rows: 4000
})

In [73]:
new_new_ds_val = new_ds["validation"].remove_columns("answers")

In [75]:
new_new_ds_val

Dataset({
    features: ['context', 'question', 'answers_generated'],
    num_rows: 400
})

In [76]:
new_new_new_ds_val = new_new_ds_val.add_column("answers", sampled2_ds["validation"]["answers"])

In [77]:
new_new_new_ds_val

Dataset({
    features: ['context', 'question', 'answers_generated', 'answers'],
    num_rows: 400
})

In [66]:
type(new_ds["validation"]["answers"])

list

In [78]:
d = DatasetDict({
    "train": new_new_new_ds,
    "validation": new_new_new_ds_val
})

In [80]:
d.push_to_hub("brimmann2/squad-v2-sampled4")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/brimmann2/squad-v2-sampled4/commit/f2107c48ba6d0e85e710b2411f65852b1a301bc5', commit_message='Upload dataset', commit_description='', oid='f2107c48ba6d0e85e710b2411f65852b1a301bc5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/brimmann2/squad-v2-sampled4', endpoint='https://huggingface.co', repo_type='dataset', repo_id='brimmann2/squad-v2-sampled4'), pr_revision=None, pr_num=None)